In [1]:
from typing import Tuple
import numpy as np
from align import NeedlemanWunsch, read_fasta


In [2]:
def _read_sub_matrix(sub_matrix_file):
        """
        DO NOT MODIFY THIS METHOD! IT IS ALREADY COMPLETE!

        This function reads in a scoring matrix from any matrix like file.
        Where there is a line of the residues followed by substitution matrix.
        This file also saves the alphabet list attribute.

        Parameters:
            sub_matrix_file: str
                Name (and associated path if not in current working directory)
                of the matrix file that contains the scoring matrix.

        Returns:
            dict_sub: dict
                Substitution matrix dictionary with tuple of the two residues as
                the key and score as value e.g. {('A', 'A'): 4} or {('A', 'D'): -8}
        """
        with open(sub_matrix_file, 'r') as f:
            dict_sub = {}  # Dictionary for storing scores from sub matrix
            residue_list = []  # For storing residue list
            start = False  # trigger for reading in score values
            res_2 = 0  # used for generating substitution matrix
            # reading file line by line
            for line_num, line in enumerate(f):
                # Reading in residue list
                if '#' not in line.strip() and start is False:
                    residue_list = [k for k in line.strip().upper().split(' ') if k != '']
                    start = True
                # Generating substitution scoring dictionary
                elif start is True and res_2 < len(residue_list):
                    line = [k for k in line.strip().split(' ') if k != '']
                    # reading in line by line to create substitution dictionary
                    assert len(residue_list) == len(line), "Score line should be same length as residue list"
                    for res_1 in range(len(line)):
                        dict_sub[(residue_list[res_1], residue_list[res_2])] = float(line[res_1])
                    res_2 += 1
                elif start is True and res_2 == len(residue_list):
                    break
        return dict_sub
test_read1 = _read_sub_matrix("substitution_matrices/BLOSUM62.mat")

def read_fasta(fasta_file: str) -> Tuple[str, str]:
    """
    DO NOT MODIFY THIS FUNCTION! IT IS ALREADY COMPLETE!

    This function reads in a FASTA file and returns the associated
    string of characters (residues or nucleotides) and the header.
    This function assumes a single protein or nucleotide sequence
    per fasta file and will only read in the first sequence in the
    file if multiple are provided.

    Parameters:
        fasta_file: str
            name (and associated path if not in current working directory)
            of the Fasta file.

    Returns:
        seq: str
            String of characters from FASTA file
        header: str
            Fasta header
    """
    assert fasta_file.endswith(".fa"), "Fasta file must be a fasta file with the suffix .fa"
    with open(fasta_file) as f:
        seq = ""  # initializing sequence
        first_header = True
        for line in f:
            is_header = line.strip().startswith(">")
            # Reading in the first header
            if is_header and first_header:
                header = line.strip()  # reading in fasta header
                first_header = False
            # Reading in the sequence line by line
            elif not is_header:
                seq += line.strip().upper()  # generating full sequence
            # Breaking if more than one header is provided in the fasta file
            elif is_header and not first_header:
                break
    return seq, header

seqA,_ = read_fasta('data/test_seq1.fa')
seqB,_ = read_fasta('data/test_seq2.fa')


In [6]:
# Assuming NeedlemanWunschV2 and blosum62 are defined as shown previously

# Define gap penalties
gap_open = -10
gap_extend = -1

# Create an instance of NeedlemanWunschV2
nw = NeedlemanWunsch(gap_open=gap_open, gap_extend=gap_extend, sub_matrix_file="substitution_matrices/BLOSUM62.mat")

# Define two sequences to align
# seqA = "AGTACG"
# seqB = "ACG"

# Perform the alignment
score, alignedA, alignedB = nw.align(seqA, seqB)

# Print the results
print(f"Alignment Score: {score}")
print(f"Aligned SeqA: {alignedA}")
print(f"Aligned SeqB: {alignedB}")

Alignment Score: 4.0
Aligned SeqA: MYQR
Aligned SeqB: M-QR


In [9]:
nw.score_matrix


array([[  0., -10., -11., -12.],
       [-10.,   5.,  -6.,  -7.],
       [-11.,  -6.,   4.,  -7.],
       [-12.,  -7.,  -1.,   5.],
       [-13.,  -8.,  -6.,   4.]])